In [ ]:
import spacy
import requests
from collections import Counter, defaultdict
import re
from datetime import datetime
import json
import os

class NorthKoreaLocationExtractor:
    def __init__(self, model_name="ko_core_news_lg"):
        """
        북한 관련 지역, 장소, 건물 정보를 추출하는 클래스
        """
        try:
            self.nlp = spacy.load(model_name)
            print(f"✅ {model_name} 모델이 성공적으로 로드되었습니다.")
        except OSError:
            print(f"❌ {model_name} 모델을 찾을 수 없습니다.")
            print("다음 명령어로 설치해주세요:")
            if model_name.startswith("ko"):
                print("pip install spacy")
                print("python -m spacy download ko_core_news_lg")
            else:
                print("pip install spacy")
                print("python -m spacy download en_core_web_sm")
            raise
            
        # JSON 파일 경로 정의
        self.location_file = '/home/ds4_sia_nolb/#FINAL_POLARIS/06_Geo_coding/Dictiionary_data/dprk_region.json'
        
        # 조합에 사용할 군사 시설 사전 경로
        self.military_facility_file = '/home/ds4_sia_nolb/#FINAL_POLARIS/06_Geo_coding/Dictiionary_data/dprk_military.json'
        
        # 다른 시설 사전 경로
        self.other_facility_files = [
            '/home/ds4_sia_nolb/#FINAL_POLARIS/06_Geo_coding/Dictiionary_data/dprk_military.json',
            '/home/ds4_sia_nolb/#FINAL_POLARIS/06_Geo_coding/Dictiionary_data/dprk_edu_cul.json',
            '/home/ds4_sia_nolb/#FINAL_POLARIS/06_Geo_coding/Dictiionary_data/dprk_landmark.json',
            '/home/ds4_sia_nolb/#FINAL_POLARIS/06_Geo_coding/Dictiionary_data/dprk_politics.json',
            '/home/ds4_sia_nolb/#FINAL_POLARIS/06_Geo_coding/Dictiionary_data/dprk_edu_cul.json'
        ]
        
        # JSON 파일에서 사전 데이터 불러오기
        self.nk_locations = self._load_json_to_set([self.location_file], "북한 지역명")
        
        # 군사 시설 사전만 별도로 로드
        self.nk_military_facilities = self._load_json_to_set([self.military_facility_file], "북한 군사 시설")
        
        # 전체 시설 사전을 통합하여 개별 단어 추출에 사용
        all_facility_files = [self.military_facility_file] + self.other_facility_files
        self.nk_facilities = self._load_json_to_set(all_facility_files, "북한 전체 시설/건물명")

        # 북한 관련 키워드 (맥락 판단용)
        self.nk_keywords = {
            '북한', '조선민주주의인민공화국', '조선', 'DPRK',
            '김정은', '김정일', '김일성', '김여정',
            '조선로동당', '노동당', '최고지도자', '원수님', '위원장',
            '핵실험', '미사일', '로켓', '인공위성', '탄도미사일',
            '대남', '남조선', '통일', '6자회담'
        }
        
        # 제외할 일반 단어들 (오탐 방지)
        self.exclude_words = {
            '해주', '순천', '개천', '성천', '신천', '안주', 
            '강계', '회창', '온천', '영원', '신원', '고원', 
            '대흥', '신양', '봉천', '송화', '과일', '신흥',
            '덕성', '영광', '고성', '철원', '평강', '김화'
        }
        
        # 맥락 패턴 (정규식)
        self.nk_context_patterns = [
            r'북한.*?([가-힣]+(?:시|군|구|동|리))', 
            r'조선.*?([가-힣]+(?:시|군|구|동|리))', 
            r'DPRK.*?([A-Za-z가-힣]+)',
            r'김정은.*?([가-힣]+(?:시|군|구|리))', 
            r'평양.*?([가-힣]+(?:구|동|리))', 
        ]

    def _load_json_to_set(self, file_paths, dict_name):
        """
        JSON 파일(들)을 읽어 세트(set)로 변환
        """
        data_set = set()
        for path in file_paths:
            if not os.path.exists(path):
                print(f"⚠️ 경고: {dict_name} 사전 파일 '{path}'을(를) 찾을 수 없습니다. 해당 사전은 비어있습니다.")
                continue
            try:
                with open(path, 'r', encoding='utf-8') as f:
                    data = json.load(f)
                    if isinstance(data, list):
                        data_set.update(data)
                    else:
                        print(f"❌ 오류: '{path}' 파일의 형식이 올바르지 않습니다. 리스트 형태여야 합니다.")
            except json.JSONDecodeError:
                print(f"❌ 오류: '{path}' 파일의 JSON 형식이 올바르지 않습니다.")
        return data_set

    def _extract_combined_locations_with_regex(self, text):
        found_combined = []

        # 지역명과 군사 시설명 패턴만 사용하여 조합
        nk_locations_pattern = '|'.join(re.escape(loc) for loc in self.nk_locations)
        nk_facilities_pattern = '|'.join(re.escape(fac) for fac in self.nk_military_facilities)

        # '지역명'과 '시설명'이 인접한 패턴을 검색 (띄어쓰기 허용)
        combined_pattern = fr'({nk_locations_pattern})\s*({nk_facilities_pattern})'
        
        sentences = re.split(r'[.!?]\s+', text)
        for sentence in sentences:
            if len(sentence.strip()) < 10:
                continue
            
            for match in re.finditer(combined_pattern, sentence):
                location = match.group(1)
                facility = match.group(2)
                combined_name = f"{location}{' ' if ' ' in sentence[match.start():match.end()] else ''}{facility}"
                
                # 중복 및 맥락 검증
                if self._is_valid_context(sentence, location):
                    found_combined.append({
                        'location': combined_name,
                        'context': sentence,
                        'confidence': 'high',
                        'type': 'combined_facility'
                    })
        
        return found_combined

    def _is_valid_context(self, sentence, location):
        """
        문맥상 해당 지역이 북한과 관련있는지 판단
        """
        sentence_lower = sentence.lower()
        location_lower = location.lower()
        
        for keyword in self.nk_keywords:
            if keyword.lower() in sentence_lower:
                return True
        
        clear_nk_locations = {'평양', '김정은', '조선로동당', '노동당'}
        for nk_loc in clear_nk_locations:
            if nk_loc in sentence and location in sentence:
                return True
        
        negative_keywords = ['한국', '남한', '우리나라', '국내', '서울', '부산']
        for neg_keyword in negative_keywords:
            if neg_keyword in sentence and location in sentence:
                return False
        
        return False

    def _extract_with_spacy_ner(self, text):
        """
        spaCy NER을 사용한 지역 추출 (개선된 버전)
        """
        doc = self.nlp(text)
        locations = []
        
        for ent in doc.ents:
            if ent.label_ in ['GPE', 'LOC']:
                location = ent.text.strip()
                sentence = ent.sent.text
                if self._is_valid_context(sentence, location):
                    locations.append({
                        'location': location,
                        'context': sentence,
                        'confidence': 'high'
                    })
        
        return locations

    def _extract_with_dictionary(self, text):
        """
        사전 기반 추출 (개선된 버전)
        """
        sentences = re.split(r'[.!?]\s+', text)
        found_locations = []
        
        for sentence in sentences:
            if len(sentence.strip()) < 10:
                continue
                
            # 북한 지역명 검색
            for location in self.nk_locations:
                if location in sentence:
                    if location in self.exclude_words:
                        if self._is_valid_context(sentence, location):
                            found_locations.append({
                                'location': location,
                                'context': sentence,
                                'confidence': 'medium',
                                'type': 'nk_location'
                            })
                    else:
                        found_locations.append({
                            'location': location,
                            'context': sentence,
                            'confidence': 'high',
                            'type': 'nk_location'
                        })
            
            # 북한 시설명 검색
            for facility in self.nk_facilities:
                if facility in sentence:
                    found_locations.append({
                        'location': facility,
                        'context': sentence,
                        'confidence': 'high',
                        'type': 'nk_facility'
                    })
        
        return found_locations

    def _filter_by_confidence(self, locations, min_confidence='medium'):
        """
        신뢰도에 따른 필터링
        """
        confidence_levels = {'low': 1, 'medium': 2, 'high': 3}
        min_level = confidence_levels.get(min_confidence, 2)
        
        filtered = []
        for loc in locations:
            loc_level = confidence_levels.get(loc.get('confidence', 'low'), 1)
            if loc_level >= min_level:
                filtered.append(loc)
        
        return filtered

    def extract_nk_locations(self, text):
        """
        텍스트에서 북한 관련 지역/장소 정보 추출 (개선된 버전)
        """
        # 1. spaCy NER 사용
        spacy_results = self._extract_with_spacy_ner(text)
        
        # 2. 사전 기반 추출 (개별 단어)
        dict_results = self._extract_with_dictionary(text)
        
        # 3. 조합된 단어 추출 (지역 + 군사시설)
        combined_results = self._extract_combined_locations_with_regex(text)
        
        # 4. 결과 통합 및 중복 제거
        all_results = spacy_results + dict_results + combined_results
        unique_locations = {}
        
        for result in all_results:
            location = result['location']
            if location not in unique_locations:
                unique_locations[location] = result
            else:
                if result.get('confidence') == 'high' and unique_locations[location].get('confidence') != 'high':
                    unique_locations[location] = result
        
        # 5. 신뢰도 필터링
        filtered_results = self._filter_by_confidence(
            list(unique_locations.values()), 
            min_confidence='medium'
        )
        
        # 6. 북한 관련 키워드 확인
        nk_context_keywords = [keyword for keyword in self.nk_keywords if keyword in text]
        
        # 7. 관련성 점수 계산
        relevance_score = self._calculate_relevance_score(text, filtered_results)
        
        return {
            'locations': filtered_results,
            'nk_context_keywords': nk_context_keywords,
            'has_nk_context': len(nk_context_keywords) > 0,
            'relevance_score': relevance_score,
            'total_found': len(filtered_results)
        }

    def _calculate_relevance_score(self, text, locations):
        """
        개선된 관련성 점수 계산
        """
        score = 0
        keyword_count = sum(1 for keyword in self.nk_keywords if keyword in text)
        score += min(keyword_count * 8, 40)
        
        high_conf_count = sum(1 for loc in locations if loc.get('confidence') == 'high')
        score += min(high_conf_count * 10, 40)
        
        med_conf_count = sum(1 for loc in locations if loc.get('confidence') == 'medium')
        score += min(med_conf_count * 5, 20)
        
        return min(score, 100)

    def analyze_news_article(self, text, title=""):
        """
        뉴스 기사를 종합적으로 분석 (개선된 버전)
        """
        full_text = f"{title} {text}" if title else text
        results = self.extract_nk_locations(full_text)
        
        summary = {
            'title': title,
            'analysis_time': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
            'text_length': len(full_text),
            'relevance_score': results['relevance_score'],
            'is_nk_related': results['has_nk_context'] and results['relevance_score'] > 30,
            'locations_found': results['locations'],
            'confidence_distribution': self._get_confidence_distribution(results['locations']),
            'full_results': results
        }
        
        return summary

    def _get_confidence_distribution(self, locations):
        """
        신뢰도별 분포 계산
        """
        distribution = {'high': 0, 'medium': 0, 'low': 0}
        for loc in locations:
            confidence = loc.get('confidence', 'low')
            distribution[confidence] += 1
        return distribution

    def print_analysis_results(self, results):
        """
        개선된 분석 결과 출력
        """
        print("=" * 60)
        print("🏴‍☠️ 북한 관련 지역·장소 분석 결과 ")
        print("=" * 60)
        
        print(f"📊 북한 관련성 점수: {results['relevance_score']}/100")
        print(f"🎯 북한 관련 기사 여부: {'✅ 예' if results['is_nk_related'] else '❌ 아니오'}")
        
        if results['locations_found']:
            print(f"\n📍 발견된 위치 정보 ({len(results['locations_found'])}개):")
            
            high_conf = [loc for loc in results['locations_found'] if loc.get('confidence') == 'high']
            med_conf = [loc for loc in results['locations_found'] if loc.get('confidence') == 'medium']
            
            if high_conf:
                print(f"\n🟢 높은 신뢰도 ({len(high_conf)}개):")
                for loc in high_conf:
                    location_type = loc.get('type', 'unknown')
                    type_icon = '🏛️' if location_type == 'nk_facility' else '🌍'
                    print(f"   {type_icon} {loc['location']} (타입: {location_type})")
                    print(f"     맥락: {loc['context'][:80]}...")
            
            if med_conf:
                print(f"\n🟡 중간 신뢰도 ({len(med_conf)}개):")
                for loc in med_conf:
                    location_type = loc.get('type', 'unknown')
                    type_icon = '🏛️' if location_type == 'nk_facility' else '🌍'
                    print(f"   {type_icon} {loc['location']} (타입: {location_type})")
                    print(f"     맥락: {loc['context'][:80]}...")
        else:
            print(f"\n❌ 북한 관련 지역/장소 정보를 찾을 수 없습니다.")
        
        if results['full_results']['nk_context_keywords']:
            print(f"\n🔑 북한 관련 키워드:")
            for keyword in results['full_results']['nk_context_keywords'][:10]:
                print(f"   • {keyword}")

def analyze_sample_file(extractor, filename="/home/ds4_sia_nolb/#FINAL_POLARIS/04_plus_preprocessing/preprocessing_final_data/re_final_preprocessing.json"):
    """
    샘플 JSON 파일을 분석하여 지역 정보 추출 (개선된 버전)
    """
    try:
        with open(filename, 'r', encoding='utf-8') as f:
            articles = json.load(f)
        
        print(f"📁 {filename} 파일에서 {len(articles)}개 기사를 읽었습니다.")
        
        extracted_results = []
        
        for i, article in enumerate(articles, 1):
            print(f"\n{'='*60}")
            print(f"📰 기사 {i}/{len(articles)}: {article['metadata']['title']}")
            print(f"🆔 ID: {article['id_']}")
            print(f"📅 날짜: {article['metadata']['pubDate']}")
            print(f"🏷️ 카테고리: {article['metadata']['category']}")
            
            full_text = f"{article['metadata']['title']} {article['text']} {article.get('summary', '')}"
            
            results = extractor.analyze_news_article(full_text, article['metadata']['title'])
            
            extractor.print_analysis_results(results)
            
            if results['locations_found'] and results['is_nk_related']:
                location_names = [loc['location'] for loc in results['locations_found']]
                extracted_results.append({
                    'id_': article['id_'],
                    'locations': location_names,
                    'confidence_info': [
                        {
                            'location': loc['location'],
                            'confidence': loc['confidence'],
                            'type': loc.get('type', 'unknown')
                        }
                        for loc in results['locations_found']
                    ]
                })
        
        print(f"\n{'='*60}")
        print("🎯 전체 분석 요약")
        print(f"{'='*60}")
        print(f"📊 분석된 기사 수: {len(articles)}개")
        print(f"📍 지역정보 발견 기사 수: {len(extracted_results)}개")
        
        if extracted_results:
            print(f"\n📋 추출된 결과 (JSON 형태):")
            print(json.dumps(extracted_results, ensure_ascii=False, indent=2))
            
            output_filename = "test_extracted_locations_improved3_2017_01_03.json"
            with open(output_filename, 'w', encoding='utf-8') as f:
                json.dump(extracted_results, f, ensure_ascii=False, indent=2)
            print(f"\n💾 결과가 {output_filename} 파일로 저장되었습니다.")
        else:
            print("\n❌ 북한 관련 지역 정보가 발견된 기사가 없습니다.")
            
        return extracted_results
        
    except FileNotFoundError:
        print(f"❌ 오류: {filename} 파일을 찾을 수 없습니다.")
        print("파일이 Python 스크립트와 같은 디렉터리에 있는지 확인해주세요.")
        return []
    except json.JSONDecodeError:
        print(f"❌ 오류: {filename} 파일의 JSON 형식이 올바르지 않습니다.")
        return []
    except Exception as e:
        print(f"❌ 오류: {e}")
        return []

def main():
    """
    메인 실행 함수
    """
    print("🏴‍☠️ 북한 관련 지역·장소 추출기 (개선 버전)")
    print("=" * 60)
    
    try:
        extractor = NorthKoreaLocationExtractor("ko_core_news_lg")
    except:
        print("\n중형 모델로 시도해보겠습니다...")
        try:
            extractor = NorthKoreaLocationExtractor("ko_core_news_md")
        except:
            print("\n소형 모델로 시도해보겠습니다...")
            try:
                extractor = NorthKoreaLocationExtractor("ko_core_news_sm")
            except:
                print("\n영어 모델로 시도해보겠습니다...")
                try:
                    extractor = NorthKoreaLocationExtractor("en_core_web_sm")
                except:
                    print("spaCy 모델을 설치한 후 다시 실행해주세요.")
                    return
    
    print("\n🔄 샘플 파일 분석을 시작합니다...")
    extracted_results = analyze_sample_file(extractor)
    
    if extracted_results:
        print(f"\n✅ 분석 완료! {len(extracted_results)}개 기사에서 신뢰할 수 있는 지역 정보를 추출했습니다.")
    else:
        print("\n⚠️  분석은 완료되었지만 신뢰할 수 있는 북한 관련 지역 정보가 발견되지 않았습니다.")

if __name__ == "__main__":
    main()

In [2]:
import spacy
from spacy.tokens import Doc
from spacy.pipeline import EntityRuler
import requests
from collections import Counter, defaultdict
import re
from datetime import datetime
import json
import os
from flashtext import KeywordProcessor

class NorthKoreaLocationExtractor:
    def __init__(self, model_name="ko_core_news_lg"):
        """
        북한 관련 지역, 장소, 건물 정보를 추출하는 클래스
        """
        try:
            spacy.prefer_gpu()
            print("✅ GPU 사용을 시도합니다.")
        except Exception as e:
            print(f"❌ GPU 활성화 실패: {e}")
            print("CPU를 사용합니다.")

        try:
            self.nlp = spacy.load(model_name)
            if self.nlp.meta['name'] == model_name:
                if spacy.prefer_gpu():
                    print(f"✅ {model_name} 모델이 GPU에서 성공적으로 로드되었습니다.")
                else:
                    print(f"✅ {model_name} 모델이 CPU에서 성공적으로 로드되었습니다.")
        except OSError:
            print(f"❌ {model_name} 모델을 찾을 수 없습니다.")
            print("다음 명령어로 설치해주세요:")
            if model_name.startswith("ko"):
                print("pip install spacy")
                print("python -m spacy download ko_core_news_lg")
            else:
                print("pip install spacy")
                print("python -m spacy download en_core_web_sm")
            raise

        # JSON 파일 경로 정의
        self.location_file = '/home/ds4_sia_nolb/#FINAL_POLARIS/06_Geo_coding/Dictiionary_data/dprk_region.json'
        
        # 조합에 사용할 군사 시설 사전 경로
        self.military_facility_file = '/home/ds4_sia_nolb/#FINAL_POLARIS/06_Geo_coding/Dictiionary_data/dprk_military.json'
        
        # 다른 시설 사전 경로
        self.other_facility_files = [
            '/home/ds4_sia_nolb/#FINAL_POLARIS/06_Geo_coding/Dictiionary_data/dprk_military.json',
            '/home/ds4_sia_nolb/#FINAL_POLARIS/06_Geo_coding/Dictiionary_data/dprk_edu_cul.json',
            '/home/ds4_sia_nolb/#FINAL_POLARIS/06_Geo_coding/Dictiionary_data/dprk_landmark.json',
            '/home/ds4_sia_nolb/#FINAL_POLARIS/06_Geo_coding/Dictiionary_data/dprk_politics.json',
            '/home/ds4_sia_nolb/#FINAL_POLARIS/06_Geo_coding/Dictiionary_data/dprk_edu_cul.json'
        ]

        # JSON 파일에서 사전 데이터 불러오기
        self.nk_locations = self._load_json_to_set([self.location_file], "북한 지역명")
        self.nk_military_facilities = self._load_json_to_set([self.military_facility_file], "북한 군사 시설")
        all_facility_files = [self.military_facility_file] + self.other_facility_files
        self.nk_facilities = self._load_json_to_set(all_facility_files, "북한 전체 시설/건물명")

        # Flashtext 키워드 프로세서 초기화
        self.keyword_processor_locations = KeywordProcessor()
        self.keyword_processor_locations.add_keywords_from_list(list(self.nk_locations))
        self.keyword_processor_facilities = KeywordProcessor()
        self.keyword_processor_facilities.add_keywords_from_list(list(self.nk_facilities))

        # 북한 관련 키워드 (맥락 판단용)
        self.nk_keywords = {
            '북한', '조선민주주의인민공화국', '조선', 'DPRK',
            '김정은', '김정일', '김일성', '김여정',
            '조선로동당', '노동당', '최고지도자', '원수님', '위원장',
            '핵실험', '미사일', '로켓', '인공위성', '탄도미사일',
            '대남', '남조선', '통일', '6자회담'
        }
        
        # 제외할 일반 단어들 (오탐 방지)
        self.exclude_words = {
            '해주', '순천', '개천', '성천', '신천', '안주', 
            '강계', '회창', '온천', '영원', '신원', '고원', 
            '대흥', '신양', '봉천', '송화', '과일', '신흥',
            '덕성', '영광', '고성', '철원', '평강', '김화'
        }

    def _load_json_to_set(self, file_paths, dict_name):
        """
        JSON 파일(들)을 읽어 세트(set)로 변환
        """
        data_set = set()
        for path in file_paths:
            if not os.path.exists(path):
                print(f"⚠️ 경고: {dict_name} 사전 파일 '{path}'을(를) 찾을 수 없습니다. 해당 사전은 비어있습니다.")
                continue
            try:
                with open(path, 'r', encoding='utf-8') as f:
                    data = json.load(f)
                    if isinstance(data, list):
                        data_set.update(data)
                    else:
                        print(f"❌ 오류: '{path}' 파일의 형식이 올바르지 않습니다. 리스트 형태여야 합니다.")
            except json.JSONDecodeError:
                print(f"❌ 오류: '{path}' 파일의 JSON 형식이 올바르지 않습니다.")
        return data_set

    def _is_valid_context(self, sentence, location):
        """
        문맥상 해당 지역이 북한과 관련있는지 판단
        """
        sentence_lower = sentence.lower()
        
        for keyword in self.nk_keywords:
            if keyword.lower() in sentence_lower:
                return True
        
        return False

    def _extract_with_spacy_ner(self, doc):
        """
        spaCy NER을 사용한 지역 추출
        """
        locations = []
        for ent in doc.ents:
            if ent.label_ in ['GPE', 'LOC']:
                location = ent.text.strip()
                sentence = ent.sent.text
                if self._is_valid_context(sentence, location):
                    locations.append({
                        'location': location,
                        'context': sentence,
                        'confidence': 'high',
                        'method': 'spacy_ner'
                    })
        return locations

    def _extract_with_flashtext(self, doc):
        """
        Flashtext를 사용한 사전 기반 추출
        """
        found_locations = []
        for sentence in doc.sents:
            sentence_text = sentence.text
            if len(sentence_text.strip()) < 10:
                continue

            # 지역명 검색
            locations_in_sentence = self.keyword_processor_locations.extract_keywords(sentence_text)
            for location in set(locations_in_sentence):
                if location in self.exclude_words:
                    if self._is_valid_context(sentence_text, location):
                        found_locations.append({
                            'location': location,
                            'context': sentence_text,
                            'confidence': 'medium',
                            'type': 'nk_location',
                            'method': 'flashtext'
                        })
                else:
                    found_locations.append({
                        'location': location,
                        'context': sentence_text,
                        'confidence': 'high',
                        'type': 'nk_location',
                        'method': 'flashtext'
                    })

            # 시설명 검색
            facilities_in_sentence = self.keyword_processor_facilities.extract_keywords(sentence_text)
            for facility in set(facilities_in_sentence):
                found_locations.append({
                    'location': facility,
                    'context': sentence_text,
                    'confidence': 'high',
                    'type': 'nk_facility',
                    'method': 'flashtext'
                })
        return found_locations

    def _extract_combined_locations_with_regex(self, text):
        """
        지역명과 군사 시설명 조합 패턴 추출 (정규식 사용)
        """
        found_combined = []
        
        # Flashtext의 키워드 목록을 사용해 정규식 패턴 생성
        nk_locations_pattern = '|'.join(re.escape(loc) for loc in self.nk_locations if len(loc) > 1)
        nk_facilities_pattern = '|'.join(re.escape(fac) for fac in self.nk_military_facilities if len(fac) > 1)

        if not nk_locations_pattern or not nk_facilities_pattern:
            return []

        combined_pattern = fr'({nk_locations_pattern})\s*({nk_facilities_pattern})'
        
        sentences = re.split(r'[.!?]\s+', text)
        for sentence in sentences:
            for match in re.finditer(combined_pattern, sentence):
                location = match.group(1)
                facility = match.group(2)
                combined_name = f"{location}{' ' if ' ' in sentence[match.start():match.end()] else ''}{facility}"
                
                if self._is_valid_context(sentence, location):
                    found_combined.append({
                        'location': combined_name,
                        'context': sentence,
                        'confidence': 'high',
                        'type': 'combined_facility',
                        'method': 'regex'
                    })
        return found_combined

    def extract_nk_locations_from_doc(self, doc):
        """
        spaCy Doc 객체에서 북한 관련 지역/장소 정보 추출
        """
        # 1. spaCy NER 사용
        spacy_results = self._extract_with_spacy_ner(doc)
        
        # 2. Flashtext 기반 사전 추출
        dict_results = self._extract_with_flashtext(doc)
        
        # 3. 조합된 단어 추출 (정규식)
        combined_results = self._extract_combined_locations_with_regex(doc.text)
        
        all_results = spacy_results + dict_results + combined_results
        unique_locations = {}
        
        for result in all_results:
            location = result['location']
            if location not in unique_locations:
                unique_locations[location] = result
            else:
                if result.get('confidence') == 'high' and unique_locations[location].get('confidence') != 'high':
                    unique_locations[location] = result
        
        return list(unique_locations.values())
    
    def _calculate_relevance_score(self, text, locations):
        """
        관련성 점수 계산
        """
        score = 0
        keyword_count = sum(1 for keyword in self.nk_keywords if keyword in text)
        score += min(keyword_count * 8, 40)
        
        high_conf_count = sum(1 for loc in locations if loc.get('confidence') == 'high')
        score += min(high_conf_count * 10, 40)
        
        med_conf_count = sum(1 for loc in locations if loc.get('confidence') == 'medium')
        score += min(med_conf_count * 5, 20)
        
        return min(score, 100)

    def analyze_results(self, full_text, locations_found, title=""):
        """
        분석 결과를 요약
        """
        relevance_score = self._calculate_relevance_score(full_text, locations_found)
        is_nk_related = (len([kw for kw in self.nk_keywords if kw in full_text]) > 0 and relevance_score > 30) or len(locations_found) > 0
        
        return {
            'title': title,
            'analysis_time': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
            'relevance_score': relevance_score,
            'is_nk_related': is_nk_related,
            'locations_found': locations_found,
        }

def process_articles_in_batches(extractor, articles, output_filename="re_extracted_locations_ten_year_all.jsonl", batch_size=1000):
    """
    기사를 배치 단위로 처리하고 JSONL 형식으로 실시간 저장
    """
    print(f"\n🔄 총 {len(articles)}개 기사를 배치 단위로 분석합니다...")
    
    start_time = datetime.now()
    processed_count = 0
    
    with open(output_filename, 'w', encoding='utf-8') as outfile:
        # 텍스트 추출 및 배치 생성
        texts = [f"{article['metadata']['title']} {article['text']} {article.get('summary', '')}" for article in articles]
        
        for i, doc in enumerate(extractor.nlp.pipe(texts, batch_size=batch_size), 1):
            article_index = i - 1
            article = articles[article_index]
            
            locations_found = extractor.extract_nk_locations_from_doc(doc)
            
            summary = extractor.analyze_results(doc.text, locations_found, article['metadata']['title'])
            
            if summary['locations_found'] and summary['is_nk_related']:
                result_entry = {
                    'id_': article['id_'],
                    'locations': [loc['location'] for loc in summary['locations_found']],
                    'confidence_info': [
                        {
                            'location': loc['location'],
                            'confidence': loc['confidence'],
                            'type': loc.get('type', 'unknown'),
                            'method': loc.get('method', 'unknown')
                        }
                        for loc in summary['locations_found']
                    ]
                }
                outfile.write(json.dumps(result_entry, ensure_ascii=False) + '\n')
            
            processed_count += 1
            if processed_count % batch_size == 0 or processed_count == len(articles):
                elapsed_time = (datetime.now() - start_time).total_seconds()
                print(f"📊 {processed_count}/{len(articles)}개 기사 분석 완료. (경과 시간: {elapsed_time:.2f}초)")

    end_time = datetime.now()
    print(f"\n✅ 전체 분석 완료! 총 {len(articles)}개 기사 분석에 { (end_time - start_time).total_seconds():.2f}초 소요되었습니다.")
    print(f"💾 결과가 {output_filename} 파일로 저장되었습니다.")
    
def main():
    """
    메인 실행 함수
    """
    print("🏴‍☠️ 북한 관련 지역·장소 추출기 (최적화 버전)")
    print("=" * 60)
    
    try:
        extractor = NorthKoreaLocationExtractor("ko_core_news_lg")
    except:
        print("\n모델 로드에 실패했습니다. 다른 모델로 시도합니다...")
        try:
            extractor = NorthKoreaLocationExtractor("ko_core_news_md")
        except:
            print("\n다른 모델 로드에도 실패했습니다. spaCy 모델을 설치한 후 다시 실행해주세요.")
            return

    input_filename = "/home/ds4_sia_nolb/#FINAL_POLARIS/04_plus_preprocessing/preprocessing_final_data/re_final_preprocessing.json"
    
    try:
        with open(input_filename, 'r', encoding='utf-8') as f:
            articles = json.load(f)
        print(f"📁 {input_filename} 파일에서 {len(articles)}개 기사를 읽었습니다.")
    except FileNotFoundError:
        print(f"❌ 오류: {input_filename} 파일을 찾을 수 없습니다.")
        return
    except json.JSONDecodeError:
        print(f"❌ 오류: {input_filename} 파일의 JSON 형식이 올바르지 않습니다.")
        return

    process_articles_in_batches(extractor, articles)

if __name__ == "__main__":
    main()

🏴‍☠️ 북한 관련 지역·장소 추출기 (최적화 버전)
✅ GPU 사용을 시도합니다.
📁 /home/ds4_sia_nolb/#FINAL_POLARIS/04_plus_preprocessing/preprocessing_final_data/re_final_preprocessing.json 파일에서 80434개 기사를 읽었습니다.

🔄 총 80434개 기사를 배치 단위로 분석합니다...
📊 1000/80434개 기사 분석 완료. (경과 시간: 40.00초)
📊 2000/80434개 기사 분석 완료. (경과 시간: 62.72초)
📊 3000/80434개 기사 분석 완료. (경과 시간: 86.30초)
📊 4000/80434개 기사 분석 완료. (경과 시간: 109.16초)
📊 5000/80434개 기사 분석 완료. (경과 시간: 133.76초)
📊 6000/80434개 기사 분석 완료. (경과 시간: 159.09초)
📊 7000/80434개 기사 분석 완료. (경과 시간: 183.98초)
📊 8000/80434개 기사 분석 완료. (경과 시간: 207.82초)
📊 9000/80434개 기사 분석 완료. (경과 시간: 231.85초)
📊 10000/80434개 기사 분석 완료. (경과 시간: 257.49초)
📊 11000/80434개 기사 분석 완료. (경과 시간: 281.12초)
📊 12000/80434개 기사 분석 완료. (경과 시간: 305.94초)
📊 13000/80434개 기사 분석 완료. (경과 시간: 330.40초)
📊 14000/80434개 기사 분석 완료. (경과 시간: 355.00초)
📊 15000/80434개 기사 분석 완료. (경과 시간: 379.63초)
📊 16000/80434개 기사 분석 완료. (경과 시간: 404.74초)
📊 17000/80434개 기사 분석 완료. (경과 시간: 431.01초)
📊 18000/80434개 기사 분석 완료. (경과 시간: 456.66초)
📊 19000/80434개 기사 분석 완료. (경과 시간: 481.58초)
📊